In [14]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

# 1. Подготавливаем webdriver

In [15]:
#Service(ChromeDriverManager().install())

In [16]:
driver = webdriver.Chrome()
# Ждем, пока браузер загрузится
time.sleep(20)

# 2. Собираем список ноутбуков

Объявляем вспомогательную функцию для скрапинга раздела со списком ноутбуков

In [17]:
def scrap_laptop_list(driver, depth=10, verbose=True):
    laptop_list = pd.DataFrame(columns=['id', 'brand', 'product', 'price', 'rating', 'votes'])
    
    for i in range(1, depth + 1):
        if verbose:
            print('Scraping page', i)
            
        # Запрашиваем страницу с порядковым номером i
        url = 'https://www.wildberries.ru/catalog/elektronika/noutbuki-pereferiya/noutbuki-ultrabuki?sort=popular&page=' + str(i)
        driver.get(url)
        # Ждем 15 секунд, чтобы страница успела загрузиться
        time.sleep(15)
        
        # Вытаскиваем нужный div со списком ноутбуков и передаем парсеру
        element = driver.find_element(By.XPATH, "//div[@class='product-card-list']")
        html = element.get_attribute('outerHTML')
        bs = BeautifulSoup(html, 'html.parser')
        
        # Парсим список ноутбуков
        pcl_tag = bs.find('div', class_='product-card-list')
        for element in pcl_tag:
            if element.name == 'div':
                                    
                # Вытаскиваем ID ноутбука
                id = int(element['data-popup-nm-id'])
                #print(id)
        
                # Вытаскиваем название бренда
                brand_tag = element.find('strong', class_='brand-name')
                brand = brand_tag.text
                #print(brand)
        
                # Вытаскиваем краткое описание ноутбука
                product_tag = element.find('span', class_='goods-name')
                product = product_tag.text.replace('Ноутбук ', '').replace(',', '')
                #print(product)
                
                # Вытаскиваем цену
                price_tag = element.find('ins', class_='price__lower-price')
                if price_tag is None:
                    price_tag = element.find('span', class_='price__lower-price')
                price = int(price_tag.text.replace(' ', '').replace('₽', '').replace(u'\xa0', ''))
                #print(price)

                # Вытаскиваем рейтинг
                rating = 0
                for r in range(0, 6):
                    if element.find('span', class_='product-card__rating stars-line star' + str(r)) is not None:
                        rating = r
                        break
                #print(rating)
                
                # Вытаскиваем число проголосовавших
                votes_tag = element.find('span', class_='product-card__count')
                votes = int(votes_tag.text)
                #print(votes)
                        
                laptop_list.loc[len(laptop_list.index)] = [
                    id,
                    brand,
                    product,
                    price,
                    rating,
                    votes
                ]
        
    return laptop_list

In [18]:
laptop_list = scrap_laptop_list(driver)

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10


In [19]:
laptop_list

,id,brand,product,price,rating,votes
0,103561157,Irbis,"NB283 14"" N3350 14"" 1366*768 IPS",13738,5,68
1,132248430,Realme,Intel Core i5 2.4ГГц 16ГБ 512ГБ,67191,5,8
2,139308234,Acer,Nitro 5 AN515-45,64061,5,1
3,133573452,Irbis,"NB290 15.6"" N4020 3200x1800 IPS 4+128G",16661,5,23
4,134148789,lenovo,IdeaPad 3 15IML05,41862,3,2
...,...,...,...,...,...,...
926,81972518,lenovo,ноудбуки,42737,0,0
927,96065157,Daurenovna,Lenovo V14-ADA 82C6S03900 серый,37890,0,0
928,96146919,CHUWI,GemiBook 8Gb 256Gb 4C4T2,35856,0,0
929,133744288,lenovo,IdeaPad L3 Celeron 6305 4ГБ 1000HDD,34990,0,0


Выгружаем список ноутбуков

In [20]:
laptop_list.to_csv('wb_laptop_list.csv', index=False)

# 3. Собираем характеристики ноутбуков

В процессе...

# 4. Закрываем webdriver

In [21]:
driver.quit()